In [28]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [29]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [30]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ai4all-project-474922')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [34]:
import os, pandas as pd
import geemap

# --- Cell 2: Settings ---
# Date ranges
ranges = [
    ("2021-10-01", "2021-10-14"),
    ("2022-11-01", "2022-11-24"),
    ("2023-03-01", "2023-06-14"),
    ("2023-08-01", "2023-11-14"),
    ("2024-03-01", "2024-06-14"),
    ("2024-08-01", "2024-11-14"),
    ("2025-03-01", "2025-06-14"),
    ("2025-08-01", "2025-10-13"),
]

# Local output folders
base_dir = "/content/viirs_data"
vt_dir, nj_dir = [os.path.join(base_dir, s) for s in ["VT", "NJ"]]
os.makedirs(vt_dir, exist_ok=True)
os.makedirs(nj_dir, exist_ok=True)

# Image collection
viirs = ee.ImageCollection("NASA/VIIRS/002/VNP46A2").select("DNB_BRDF_Corrected_NTL")

# Regions
vt = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("STUSPS", "VT"))
nj = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("STUSPS", "NJ"))

# Function to get mean nightly radiance
def get_viirs_mean(start, end, region, state_code):
    imgs = viirs.filterDate(start, end)
    dates = ee.List(imgs.aggregate_array("system:time_start")).map(lambda d: ee.Date(d).format("YYYY-MM-dd"))
    return imgs, dates.getInfo()

# --- Cell 3: Download function ---
def download_viirs_images(state_fc, state_code, out_dir):
    records = []
    for start, end in ranges:
        imgs, date_list = get_viirs_mean(start, end, state_fc, state_code)
        ic = viirs.filterDate(start, end)
        # iterate through each date
        for d in date_list:
            img = ic.filterDate(d, ee.Date(d).advance(1, "day")).mean().clip(state_fc)
            name = f"{state_code}_{d}.tif"
            path = os.path.join(out_dir, name)
            print(f"Downloading {name} ...")
            geemap.ee_export_image(
                img,
                filename=path,
                scale=500,
                region=state_fc.geometry(),
                file_per_band=False
            )
            records.append({"date": d, "filename": name})
    # Save metadata
    df = pd.DataFrame(records)
    csv_path = os.path.join(out_dir, f"{state_code}_metadata.csv")
    df.to_csv(csv_path, index=False)
    print(f"Saved metadata to {csv_path}")

# --- Cell 4: Run downloads ---
download_viirs_images(vt, "VT", vt_dir)
download_viirs_images(nj, "NJ", nj_dir)


Streaming output truncated to the last 5000 lines.
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-05.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-06.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-07.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-08.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-09.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-10.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-11.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-12.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-13.tif
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2024-03-14.tif
Generating URL ...

In [37]:
!zip -r viirs_data.zip /content/viirs_data
from google.colab import files
files.download("viirs_data.zip")

  adding: content/viirs_data/ (stored 0%)
  adding: content/viirs_data/NJ/ (stored 0%)
  adding: content/viirs_data/NJ/NJ_2025-06-11.tif (deflated 1%)
  adding: content/viirs_data/NJ/NJ_2025-08-04.tif (deflated 1%)
  adding: content/viirs_data/NJ/NJ_2025-04-01.tif (deflated 93%)
  adding: content/viirs_data/NJ/NJ_2025-04-08.tif (deflated 7%)
  adding: content/viirs_data/NJ/NJ_2023-06-06.tif (deflated 1%)
  adding: content/viirs_data/NJ/NJ_2023-04-27.tif (deflated 6%)
  adding: content/viirs_data/NJ/NJ_2023-10-28.tif (deflated 1%)
  adding: content/viirs_data/NJ/NJ_2024-09-18.tif (deflated 93%)
  adding: content/viirs_data/NJ/NJ_2022-11-02.tif (deflated 1%)
  adding: content/viirs_data/NJ/NJ_2023-10-22.tif (deflated 1%)
  adding: content/viirs_data/NJ/NJ_2024-03-15.tif (deflated 50%)
  adding: content/viirs_data/NJ/NJ_2023-10-29.tif (deflated 3%)
  adding: content/viirs_data/NJ/NJ_2023-05-19.tif (deflated 1%)
  adding: content/viirs_data/NJ/NJ_2023-10-30.tif (deflated 11%)
  adding: con

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>